# Lecture 20 - Linear Systems
 
In this lecture, we consider the linear system $\mathbf{Ax}=\mathbf{b}$, where $\mathbf{A}$ is a square $n\times n$ matrix, while both $\mathbf{x}$ and $\mathbf{b}$ are vectors of $n$ elements.  We'll examine the basic *elimination* algorithm and review how to set up and solve such systems with NumPy.

### Objectives

By the end of this lesson, you should be able to

- Explain what it means for $\mathbf{Ax}=\mathbf{b}$ to have a solution.
- Define $\mathbf{Ax}=\mathbf{b}$ from engineering or other inputs.
- Solve linear systems using basic Gaussian elimination.
- Solve linear systems using NumPy.

### Key Terms

- matrix
- vector
- matrix-vector product
- column space
- Gaussian elimination
- pivot
- `np.linalg.solve`

## Matrix-Vector Products and Relation to Linear Systems

A fundamental operation in linear algebra is the *matrix-vector* product $\mathbf{Av} \to \mathbf{w}$.

Example for $3\times 3$ system:
$$
\mathbf{Av} = 
\begin{bmatrix} 
  a_{00} & a_{01} & a_{02} \\
  a_{10} & a_{11} & a_{12} \\
  a_{20} & a_{21} & a_{22} \\
\end{bmatrix} 
\times
\left[ \begin{array}{c} 
   v_0 \\ 
   v_1 \\
   v_2
   \end{array} \right] =
\left[ \begin{array}{c} 
   a_{00} v_0 + a_{01} v_1 + a_{02} v_2 \\ 
   a_{10} v_0 + a_{11} v_1 + a_{12} v_2 \\ 
   a_{20} v_0 + a_{21} v_1 + a_{22} v_2 \\ 
   \end{array} \right] 
= \mathbf{w} \, .
$$

Each element of $\mathbf{w}$ is a *dot product* or *inner product*:

$$
  w_i = \sum_{j=0}^{j=n-1} a_{ij} v_j \, = \mathbf{a}_{i:}^T \mathbf{v} \, .
$$

In [10]:
def matrix_vector(A, v):
    """Matrix-vector product for a square matrix A."""
    n = len(v)
    w = np.zeros_like(v)
    for i in range(n):
        for j in range(n):
            w[i] += A[i, j]*v[j] # or replace 6&7 by w[i] = A[i, :].dot(v)
    return w

**Exercise**:  What *order* is this matrix-vector algorithm?  In other words, if we increase $n$ from $10$ to $100$, by how much would the time to execute `matrix_vector` increase?  Here, the unit of work is the product `A[i, j]*v[j]`, which is one *floating-point operation* or *FLOP*.

In [20]:
import numpy as np
from time import time
n = 10
A, v = np.ones((n, n)), np.ones(n)
t0 = time()
w = matrix_vector(A, v)
te = time() - t0
print('elapsed time = {:.2e} seconds'.format(te))

elapsed time = 1.55e-04 seconds


An *alternative* view of $\mathbf{Av} \to \mathbf{w}$:

$$
  \mathbf{w} = 
v_0 \left[ \begin{array}{c} 
   a_{00} \\ 
   a_{10} \\
   a_{20}
   \end{array} \right] +
v_1 \left[ \begin{array}{c} 
   a_{01} \\ 
   a_{11} \\
   a_{21}
   \end{array} \right] +
v_2 \left[ \begin{array}{c} 
   a_{02} \\ 
   a_{12} \\
   a_{22}
   \end{array} \right] 
=  v_0 \mathbf{a}_{:0} + v_1\mathbf{a}_{:1} + v_2\mathbf{a}_{:2} \, .
$$

Hence, $\mathbf{w}$ is a *linear combination of the columns* of $\mathbf{A}$, and all possible linear combinations of $\mathbf{A}$'s represents the *column space* of $\mathbf{A}$.

The *connection*: left and right sides of $\mathbf{Ax}=\mathbf{b}$ equal if and only if $\mathbf{b}$ is a linear combination of the columns of $\mathbf{A}$:

$$
\begin{split}
\mathbf{Ax} & = 
\begin{bmatrix} 
  a_{00} & a_{01} & \cdots & a_{0,n-1} \\
  a_{10} & a_{11} & \cdots & a_{1,n-1} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{m-1,0} & a_{m-1,2} & \cdots & a_{m-1,n-1} 
\end{bmatrix} 
\times 
\left[ \begin{array}{c} 
   x_0 \\ 
   x_1 \\
   \vdots \\
   x_{m-1} 
   \end{array} \right] \\
& = x_0 \mathbf{a}_{:, 0} + x_1 \mathbf{a}_{:, 1} + \ldots + x_{m-1} \mathbf{a}_{:,m-1}  \\
& = \mathbf{b}
\end{split}
$$

## Elimination and Substitution for Solving Systems

Consider the following (slightly modified from the reading:

In [52]:
import numpy as np
def eliminate(A, b):
    U, c = np.copy(A), np.copy(b)
    for j in range(len(U)):
        pivot = U[j, j]
        assert pivot != 0.0, "pivot is zero!"
        for i in range(j+1, len(A)):
            multiple = U[i, j]/pivot
            c[i] = c[i] - c[j]*multiple
            for k in range(j, len(A)):
                U[i, k] = U[i, k] - U[j, k]*multiple
        print('column = {}, pivot = {:.4f}'.format(j, pivot))
        s = "U = \n{}".format(U)
        s = s.replace('[[',' [').replace(']]', ']')
        s = s.replace('[','|').replace(']',' |')
        print(s)
    return U, c

In [55]:
A = np.array([[ 2.0, -1.0,  0.0],
              [-1.0,  2.0, -1.0],
              [ 0.0, -1.0,  2.0]])
b = np.array([1.0, 1.0, 1.0])
U, c = eliminate(A, b)

column = 0, pivot = 2.0000
U = 
 | 2.  -1.   0.  |
 | 0.   1.5 -1.  |
 | 0.  -1.   2.  |
column = 1, pivot = 1.5000
U = 
 | 2.         -1.          0.         |
 | 0.          1.5        -1.         |
 | 0.          0.          1.33333333 |
column = 2, pivot = 1.3333
U = 
 | 2.         -1.          0.         |
 | 0.          1.5        -1.         |
 | 0.          0.          1.33333333 |


**Exercise**: Can you reason out *a priori* what the order of elimination is?  Perform a numerical experiment to check your prediction.

Elimination is the hard part, but we do not yet have the solution $\mathbf{x}$. 

The *solution*:  use *backsubstitution* by solving for $x_{n-1}$ first, then using it to find $x_{n-2}$, etc.

In [63]:
def back_substitute(U, c):
    m = len(c)
    x = np.zeros_like(c)
    for i in range(m-1, -1, -1):
        # Add up all the pieces that go on the right hand side
        rhs = c[i] 
        j = i + 1 
        for j in range(i+1, m):
            rhs = rhs - U[i, j]*x[j] # Recall that x[j] is known!
        x[i] = rhs / U[i, i] # Solve for the next unknown
    return x

In [64]:
back_substitute(U, c)

array([ 1.5,  2. ,  1.5])

## Practical Use of Matrices

<img src="small_truss.png" alt="Truss" style="width: 300px;"/>

## Recap

You should now be able to

- Explain what it means for $\mathbf{Ax}=\mathbf{b}$ to have a solution.
- Define $\mathbf{Ax}=\mathbf{b}$ from engineering or other inputs.
- Solve linear systems using basic Gaussian elimination.
- Solve linear systems using NumPy.